In [20]:
import os
import pandas as pd
import openpyxl

# Define the directory where your CSV files are located
csv_directory = 'csvs'  # Change this to the directory containing your CSV files

# Initialize an empty dictionary to store DataFrames
data_frames = {}

df = pd.read_excel('GENeSYS-MOD_User_Input_Settings_v01_phe_05-09-2023.xlsx', sheet_name='Region_selection')
filtered_df = df[df.iloc[:, -1] == 1]
unique_values_from_first_df = filtered_df.iloc[:, 3].unique()

# Initialize the Excel writer
with pd.ExcelWriter('output.xlsx', engine='openpyxl') as writer:
    for filename in os.listdir(csv_directory):
        if filename.endswith('.csv'):
            # Construct the full path to the CSV file
            csv_file_path = os.path.join(csv_directory, filename)
            
            # Read the CSV file into a Pandas DataFrame, skipping the first 4 rows
            df = pd.read_csv(csv_file_path, skiprows=4, delimiter=';')
            
            # Extract values from the first column
            first_column_values = df.iloc[:, 0]
            
            # Filter the DataFrame based on whether the values in the first column are present in unique_values_from_first_df
            filtered_df = df[first_column_values.isin(unique_values_from_first_df)]
            
            # Get the worksheet name without the .csv extension
            worksheet_name = os.path.splitext(filename)[0]
            
            # Store the DataFrame in the dictionary with the filename as the key
            data_frames[worksheet_name] = filtered_df
            
            # Write the DataFrame to an Excel worksheet with the same filename (without extension)
            filtered_df.to_excel(writer, sheet_name=worksheet_name, index=False)


